In [1]:
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import InceptionV3,preprocess_input,decode_predictions
from keras.preprocessing import image
from keras.optimizers import SGD
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
import numpy as np

Using TensorFlow backend.


In [2]:
img_height=299
img_width=299

In [3]:
train_data_dir = 'images/train'
validation_data_dir = 'images/valid'
test_data_dir = 'images/test'

In [4]:
base_model = InceptionV3(weights='imagenet',include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)

predictions=Dense(1,activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

In [5]:
for layer in base_model.layers:
  layer.trainable = False

In [6]:
model.compile(optimizer='adam', metrics=['accuracy'], loss='binary_crossentropy')

In [7]:
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2,zoom_range=0.2, horizontal_flip=True)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_data_dir, target_size=(img_height,img_width), batch_size=20, class_mode='binary')
validation_generator = validation_datagen.flow_from_directory(validation_data_dir, target_size=(img_height,img_width), batch_size=20, class_mode='binary')
test_generator = test_datagen.flow_from_directory(test_data_dir, target_size=(img_height,img_width), batch_size=20, class_mode='binary')


Found 800 images belonging to 2 classes.
Found 200 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


In [8]:
model.fit_generator( train_generator, steps_per_epoch=200,validation_data=validation_generator, validation_steps=10, epochs=2)

Epoch 1/2
200/200 [==============================] - 145s 723ms/step - loss: 0.2721 - acc: 0.8875 - val_loss: 0.0568 - val_acc: 0.9750
Epoch 2/2
200/200 [==============================] - 134s 670ms/step - loss: 0.1448 - acc: 0.9447 - val_loss: 0.0947 - val_acc: 0.9550


In [9]:
for layer in model.layers[:172]:
  layer.trainable= False
for layer in model.layers[172:]:
  layer.trainable= True

In [10]:
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), metrics=['accuracy'], loss='binary_crossentropy')

In [11]:
model.fit_generator( train_generator, steps_per_epoch=200,validation_data=validation_generator, validation_steps=10, epochs=2)

Epoch 1/2
200/200 [==============================] - 164s 818ms/step - loss: 0.0926 - acc: 0.9677 - val_loss: 0.1245 - val_acc: 0.9300
Epoch 2/2
200/200 [==============================] - 174s 872ms/step - loss: 0.0766 - acc: 0.9732 - val_loss: 0.1275 - val_acc: 0.9250


In [14]:
#preds=model.predict(test_datagen)

In [15]:
test_generator

In [ ]:
type